In [1]:
#set up
import pandas as pd
import requests
import json
import sqlalchemy as sa 

#import env variables
from dotenv import load_dotenv
load_dotenv()

import os
airlab_api = os.environ['AIRLABS_API_KEY']

#get connection
from gans_connection import get_database_connection

In [2]:
def fetch_arrival_info(iata_code):
    try:
        api_url = f"https://airlabs.co/api/v9/schedules?arr_iata={iata_code}&limit=40&api_key={airlab_api}"
       
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            return data
        
    except Exception as e:
        print(f"Failed to fetch X.X = {response.status_code}")
        return None

In [3]:
def get_arrivals_infos():
    try:
        #db connection
        engine = get_database_connection()
    
        # import cities table
        cities_airports = pd.read_sql('cities_airports', engine)
    
        arrivals = []

        for index, row in cities_airports.iterrows():
            iata_code = row['airport_iata']
        
            # Fetch arrival
            data = fetch_arrival_info(iata_code)

            
            for a in data["response"]:
                #print(json.dumps(a, indent=4))
                
                arrivals_infos = { 
                    "flight_num":a["flight_number"],
                    "airline_iata": a[ "airline_iata"],
                    "departure_iata":a["dep_iata"],
                    "arrival_iata":a["arr_iata"],
                    "arrival_time":a["arr_time"],
                    "status":a["status"]
                   }
                
                # append to the list above 
                arrivals.append(arrivals_infos)   

        # Create and send table to db
        arrivals_df = pd.DataFrame(arrivals)
        arrivals_df.to_sql('flights',engine, if_exists='append', index=False)
        
    
    except Exception as e:
        print(f"Uh-oh, unexpected ARRIVALS anomaly X.X {e}")
        
    return print("Scheduled on time :D ")


In [4]:
get_arrivals_infos()

Scheduled on time :D 
